<a href="https://colab.research.google.com/github/hanghae-plus-AI/AI-1-ssungz789/blob/main/w4/transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets evaluate accelerate scikit-learn

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.9 MB/s eta 0:00:00


In [2]:
import random
import evaluate
import numpy as np

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [3]:
ag_news = load_dataset("fancyzhx/ag_news")
ag_news

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})

In [4]:
ag_news['train'][0]

{'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.",
 'label': 2}

In [5]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def preprocess_function(data):
    return tokenizer(data["text"])

ag_news_tokenized = ag_news.map(preprocess_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [6]:
ag_news_tokenized['train'][0].keys()

dict_keys(['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'])

In [7]:
ag_news_split = ag_news_tokenized['train'].train_test_split(test_size=0.2)
ag_news_train, ag_news_val = ag_news_split['train'], ag_news_split['test']
ag_news_test = ag_news_tokenized['test']

In [8]:
len(ag_news_train), len(ag_news_val), len(ag_news_test)
num_labels = len(ag_news['train'].unique('label'))

In [9]:
from transformers import BertConfig

config = BertConfig()

config.hidden_size = 128  # BERT layer의 기본 hidden dimension
config.intermediate_size = 128  # FFN layer의 중간 hidden dimension
config.num_hidden_layers = 4  # BERT layer의 개수
config.num_attention_heads = 8  # Multi-head attention에서 사용하는 head 개수
config.num_labels = num_labels  # 마지막에 예측해야 하는 분류 문제의 class 개수    -> AG News 분류 4 개

# config.hidden_dropout_prob = 0.3
# config.attention_probs_dropout_prob = 0.3

model = AutoModelForSequenceClassification.from_config(config)

In [27]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='hf_transformer',  # 모델, log 등을 저장할 directory
    num_train_epochs=15,  # epoch 수
    per_device_train_batch_size=64,  # training data의 batch size
    per_device_eval_batch_size=64,  # validation data의 batch size
    logging_strategy="epoch",
    do_train=True,
    do_eval=True,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-5,
    lr_scheduler_type="linear", # 선형 학습률 감소 사용
    load_best_model_at_end=True,
    warmup_steps=500,  # 웜업 스텝 설정
    weight_decay=0.01  # weight decay 적용
)


In [11]:
import evaluate

accuracy = evaluate.load("accuracy")


def compute_metrics(pred):
    predictions, labels = pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [12]:
from transformers import TrainerCallback

class LoggingCallback(TrainerCallback):
    def on_train_begin(self, args, state, control, **kwargs):
        print(f"학습 시작 시간: {datetime.now()}")

    def on_epoch_end(self, args, state, control, **kwargs):
        print(f"학습 완료 : {state.epoch}")
        print(f"메트릭 : {state.best_metric}")

    def on_train_end(self, args, state, control, **kwargs):
        print(f"학습 종료 시간: {datetime.now()}")

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs:
            print(f"현재 loss: {logs.get('loss', 'N/A')}")

In [13]:
from transformers import EarlyStoppingCallback
from datetime import datetime

callbacks = [
    LoggingCallback(),
    EarlyStoppingCallback(early_stopping_patience=1)
]

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ag_news_train,
    eval_dataset=ag_news_val,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    callbacks=callbacks
)

In [14]:
#trainer.train()

In [15]:
#trainer.evaluate(imdb_test)

In [16]:
#trainer.save_model()

In [17]:
#from transformers import pipeline

#classifier = pipeline("sentiment-analysis", model="./hf_transformer/", device='cuda')
#print(classifier("UK charges 8 in terror plot linked to alert in US LONDON, AUGUST 17: Britain charged eight terror suspects on Tuesday with conspiracy to commit murder and said one had plans that could be used in striking US buildings that were the focus of security scares this month."))

In [18]:
model = AutoModelForSequenceClassification.from_config(config)

In [19]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.3, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-3): 4 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.3, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-1

In [20]:
# for param in model.distilbert.parameters():
#   param.requires_grad = False

In [21]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

def preprocess_function(data):
    return tokenizer(data["text"])

ag_news_tokenized = ag_news.map(preprocess_function, batched=True)
# ag_news_split = ag_news_tokenized['train'].train_test_split(test_size=0.2)
# ag_news_train, ag_news_val = ag_news_split['train'], ag_news_split['test']
# ag_news_test = ag_news_tokenized['test']

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [28]:
callbacks = [
    LoggingCallback(),
    #EarlyStoppingCallback(early_stopping_patience=1)
]

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ag_news_train,
    eval_dataset=ag_news_val,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    callbacks=callbacks
)

In [29]:
trainer.train()
trainer.save_model()

학습 시작 시간: 2024-10-10 14:10:01.907211


Epoch,Training Loss,Validation Loss,Accuracy
1,0.327900,0.347919,0.891958
2,0.309100,0.337111,0.894500
3,0.294900,0.331594,0.897708
4,0.284000,0.326645,0.898250
5,0.276000,0.323553,0.900083
6,0.268900,0.321815,0.900667
7,0.263100,0.320710,0.900042
8,0.257000,0.318758,0.902125
9,0.252500,0.318735,0.902667
10,0.248300,0.317492,0.903125


학습 완료 : 1.0
메트릭 : None
현재 loss: 0.3279
현재 loss: N/A
학습 완료 : 2.0
메트릭 : 0.34791868925094604
현재 loss: 0.3091
현재 loss: N/A
학습 완료 : 3.0
메트릭 : 0.33711084723472595
현재 loss: 0.2949
현재 loss: N/A
학습 완료 : 4.0
메트릭 : 0.33159372210502625
현재 loss: 0.284
현재 loss: N/A
학습 완료 : 5.0
메트릭 : 0.3266453444957733
현재 loss: 0.276
현재 loss: N/A
학습 완료 : 6.0
메트릭 : 0.3235528767108917
현재 loss: 0.2689
현재 loss: N/A
학습 완료 : 7.0
메트릭 : 0.32181456685066223
현재 loss: 0.2631
현재 loss: N/A
학습 완료 : 8.0
메트릭 : 0.320710152387619
현재 loss: 0.257
현재 loss: N/A
학습 완료 : 9.0
메트릭 : 0.3187577426433563
현재 loss: 0.2525
현재 loss: N/A
학습 완료 : 10.0
메트릭 : 0.318734735250473
현재 loss: 0.2483
현재 loss: N/A
학습 완료 : 11.0
메트릭 : 0.3174915611743927
현재 loss: 0.2467
현재 loss: N/A
학습 완료 : 12.0
메트릭 : 0.3174915611743927
현재 loss: 0.2444
현재 loss: N/A
학습 완료 : 13.0
메트릭 : 0.3166525959968567
현재 loss: 0.2419
현재 loss: N/A
학습 완료 : 14.0
메트릭 : 0.3166525959968567
현재 loss: 0.24
현재 loss: N/A
학습 완료 : 15.0
메트릭 : 0.31632089614868164
현재 loss: 0.2397
현재 loss: N/A
현재 loss: N/A
학습 종료 시

여러번 학습하면서 parameter를 튜닝해보았습니다.
처음에 과제를 잘 못 보고, imdb를 90%까지 올려보려고 kfold까지 써봤지만 안되서,
코치님들을 원망..했지만 좋은 케이스가 쌓였습니다.

ag_news를 사용해도 90%에 맞추는것이 생각보다 쉽지 않았고,
85~89 언저리를 왔다갔다해서, 파라미터를 찾아보았고,
EarlyStoppingCallback를 넣는것 보다 warmup을 추가하는 것이 유의미한 데이터를 얻을 수 있겠다는 생각이 들어 추가하게 되었습니다.


12번째 학습까지는 특이 사항이 없는데, 13~15 사이에 validation loss가 높아,
overfitting을 의심했지만,아래 마지막 지시사항인 문장 예측에서도 높은 스코어를 보이고 있기도 하고 validation loss가 살짝 높아질 때 Accuracy도 같이 떨어져서 overfitting의 가능성은 낮을 것으로 예상했습니다.



In [30]:
trainer.predict(ag_news_test)

PredictionOutput(predictions=array([[ 1.2921678 , -4.87566   ,  3.1057916 , -1.145755  ],
       [-3.5063117 , -1.0291612 , -0.23614958,  3.7680273 ],
       [-3.009146  , -2.1871219 ,  0.30763555,  3.6284628 ],
       ...,
       [ 1.0897644 ,  5.5677524 , -3.5714548 , -0.73929286],
       [ 0.04956812, -5.685942  ,  3.5506895 , -0.25796175],
       [-1.5013863 , -4.9116406 ,  2.7919319 ,  1.5245632 ]],
      dtype=float32), label_ids=array([2, 3, 3, ..., 1, 2, 2]), metrics={'test_loss': 0.3313201069831848, 'test_accuracy': 0.8993421052631579, 'test_runtime': 2.3855, 'test_samples_per_second': 3185.908, 'test_steps_per_second': 49.885})

In [34]:
trainer.evaluate(ag_news_test)

현재 loss: N/A


{'eval_loss': 0.3313201069831848,
 'eval_accuracy': 0.8993421052631579,
 'eval_runtime': 2.4261,
 'eval_samples_per_second': 3132.585,
 'eval_steps_per_second': 49.05,
 'epoch': 15.0}

In [35]:
from transformers import pipeline
classifier = pipeline("sentiment-analysis", model="./hf_transformer/", device='cuda')
#print(classifier("The movie was so disgusting..."))
print(classifier("UK charges 8 in terror plot linked to alert in US LONDON, AUGUST 17: Britain charged eight terror suspects on Tuesday with conspiracy to commit murder and said one had plans that could be used in striking US buildings that were the focus of security scares this month."))

[{'label': 'LABEL_3', 'score': 0.9672237634658813}]
